<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Laboratoire 5
# Applications du modèle word2vec

**Objectifs**

Comparer des modèles word2vec pré-entraînés sur l’anglais avec des modèles appris localement sur
deux corpus, en les appliquant à des tâches de mesures de similarité et d’analogie entre mots.

Groupe: 
> François Burgener, Tiago Povoa Quinteiro

Consignes
* Soumettre sur Cyberlearn un notebook Jupyter avec les expériences, les résultats obtenus et leur
analyse. Bien présenter les étapes suivies, et répondre clairement aux questions posées dans
l’énoncé. Bien préciser les données et les commandes utilisées.
* Le travail est à effectuer en binôme.
* Ne pas hésiter à consulter la documentation de Gensim sur word2vec, ainsi que celle sur les
KeyedVectors, qui forment une classe plus générale avec plusieurs exemples intéressants.
* Les différentes tâches se feront soit sur votre propre ordinateur (si vous disposez d’au moins 16
Go de RAM), soit sur un notebook fourni par le service Google Colab.

## Partie 1. Tester et évaluer un modèle entraîné sur Google News


a. Installez gensim, la bibliothèque implémentant les outils pour travailler avec Word2Vec ( pip
install --upgrade gensim ). Puis chargez le modèle word2vec pré-entraîné sur le corpus
Google News en écrivant : w2v_model = gensim.downloader.load("word2vec-google-news-
300") , ce qui télécharge le fichier la première fois, et enfin en ne gardant que les vecteurs de
mots, avec « w2v_vectors = w2v_model.wv » puis « del w2v_model » ).
Si on dispose du fichier en local, on peut le charger en écrivant w2v_vectors =
KeyedVectors.load_word2vec_format(path_to_file, binary=True) . 

### Quelle place mémoire occupe le processus du notebook une fois les vecteurs de mots chargés ?

In [19]:
import gensim.downloader as api

In [20]:

w2v_model = api.load("word2vec-google-news-300")

In [21]:
w2v_vectors = w2v_model.wv
del w2v_model

/home/chadanlo/anaconda3/envs/tal/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


Le processus du notebook a l'air d'occuper 4.5 GiB

#### b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ? Et quelle est la taille du vocabulaire du modèle ? Identifiez cinq mots qui sont dans le vocabulaire et un qui ne l’est pas.

In [22]:
from gensim.test.utils import get_tmpfile, common_texts

fname = get_tmpfile("vectors.kv")
w2v_vectors.save(fname)

In [23]:
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load(fname, mmap='r')

In [24]:
print('Pizza: ', len(w2v_vectors['Pizza']))
print('Spaghetti: ', len(w2v_vectors['Spaghetti']))
print('Hamburger: ', len(w2v_vectors['Hamburger']))
print('Cheesecake: ', len(w2v_vectors['Cheesecake']))
print('Brownie: ', len(w2v_vectors['Brownie']))

Pizza:  300
Spaghetti:  300
Hamburger:  300
Cheesecake:  300
Brownie:  300


La dimension de l'espace vectoriel d'un vecteur semble être **300**.

In [25]:
try:
  print('COVID-19: ', len(w2v_vectors['COVID-19']))
except:
  print("Not in vocabulary") 

Not in vocabulary


In [26]:
# Vocabulary size

print("La taille est de: ", len(w2v_vectors.vocab.keys()))

La taille est de:  3000000


### c. Comment peut-on mesurer la distance entre deux mots dans cet espace ? Calculez par exemple la distance entre les mots rabbit et carrot.

In [27]:
def dist_w2v(word1, word2):
    return w2v_vectors.distance(word1, word2)

distance = dist_w2v("rabbit", "carrot")
print("{:.1f}".format(distance))

0.6


### d. Testez le modèle de distance entre mots. Est-ce que des mots proches sémantiquement sont aussi proches dans l’espace vectoriel, et inversement ? Testez au moins cinq paires de mots. 

In [28]:
from itertools import combinations

def eval_dist(words):
    comb = list(combinations(words, r=2))

    for p in comb: 
        distance = dist_w2v(p[0], p[1])
        print("Distance between: ", p, "{:.1f}".format(distance))

In [29]:
some_words = ["booze", "alcohol", "brew", "wine", "cocktail", "beer", "ethanol"]

eval_dist(some_words)

Distance between:  ('booze', 'alcohol') 0.3
Distance between:  ('booze', 'brew') 0.6
Distance between:  ('booze', 'wine') 0.6
Distance between:  ('booze', 'cocktail') 0.6
Distance between:  ('booze', 'beer') 0.4
Distance between:  ('booze', 'ethanol') 0.9
Distance between:  ('alcohol', 'brew') 0.7
Distance between:  ('alcohol', 'wine') 0.6
Distance between:  ('alcohol', 'cocktail') 0.7
Distance between:  ('alcohol', 'beer') 0.5
Distance between:  ('alcohol', 'ethanol') 0.7
Distance between:  ('brew', 'wine') 0.5
Distance between:  ('brew', 'cocktail') 0.5
Distance between:  ('brew', 'beer') 0.4
Distance between:  ('brew', 'ethanol') 0.8
Distance between:  ('wine', 'cocktail') 0.6
Distance between:  ('wine', 'beer') 0.4
Distance between:  ('wine', 'ethanol') 0.8
Distance between:  ('cocktail', 'beer') 0.6
Distance between:  ('cocktail', 'ethanol') 1.0
Distance between:  ('beer', 'ethanol') 0.7


Alcohol et ethanol sont techniquement la même chose, mais ont une distance très élevée.
Le sens scientifique éloigne probablement. 

Wine, beer et booze sont proches. Brew et beer aussi. 

### e. Y a-t-il des cas ambigus, et pourquoi selon vous ? Par exemple, pouvez-vous trouver des mots opposés selon le sens qui sont proches dans l’espace réduit ?

In [30]:
other_words = ["joy", "sadness", "happiness", "mournful", "delight"]


eval_dist(other_words)

Distance between:  ('joy', 'sadness') 0.4
Distance between:  ('joy', 'happiness') 0.4
Distance between:  ('joy', 'mournful') 0.7
Distance between:  ('joy', 'delight') 0.3
Distance between:  ('sadness', 'happiness') 0.6
Distance between:  ('sadness', 'mournful') 0.6
Distance between:  ('sadness', 'delight') 0.6
Distance between:  ('happiness', 'mournful') 0.8
Distance between:  ('happiness', 'delight') 0.7
Distance between:  ('mournful', 'delight') 0.9


Joy est à la même distance de sadness et happiness. Rigolo :)

L'exemple ci-dessus démontre quelques cas amusants.

Joy est proche de happiness et delight, mais entre eux happiness et delight sont très éloignés

### f. Que dire des mots ayant plusieurs sens ? Pouvez-vous donner 3 exemples de ce problème ?

In [31]:
other_words = ["run", "sprint", "escape", "jog", "start"]


eval_dist(other_words)

Distance between:  ('run', 'sprint') 0.6
Distance between:  ('run', 'escape') 0.8
Distance between:  ('run', 'jog') 0.7
Distance between:  ('run', 'start') 0.6
Distance between:  ('sprint', 'escape') 0.9
Distance between:  ('sprint', 'jog') 0.6
Distance between:  ('sprint', 'start') 0.8
Distance between:  ('escape', 'jog') 0.8
Distance between:  ('escape', 'start') 0.9
Distance between:  ('jog', 'start') 0.8


Run est réputé pour avoir plusieurs signifiés.

Ici on voit pourtant que run n'est proche d'aucun de ces synonymes. 

source: https://www.thesaurus.com/browse/run?s=t

### g. En vous aidant de la documentation de Gensim sur KeyedVectors, mesurez de manière quantitative la performance du modèle sur le corpus WordSimilarity-353. Expliquez ce que signifient vos résultats.

In [32]:
from gensim.test.utils import datapath

In [33]:
similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

similarities

((0.6238773466616107, 1.7963237724171284e-39),
 SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45),
 0.0)

Returns
* pearson (tuple of (float, float)) – Pearson correlation coefficient with 2-tailed p-value.
* spearman (tuple of (float, float)) – Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value.
* oov_ratio (float) – The ratio of pairs with unknown words.

Comme on peut voir au-dessus:
* Pearson: 0.62, 1.79e-39
    * Décrit la correlation linéaire entre deux variables. Si elle vaut 0, il n'y a pas de correlation (On ne pourrait pas faire de régression linéaire là-dessus?). La valeur est entre -1 et 1. Ici on peut dire que c'est assez fortement correlé positivement. 
    * pvalue=> Marge d'erreur très faible à 10^-39
* Spearman corelation=0.66, pvalue=2.53e-45
    * Mesure différente mais à l'objectif similaire à Pearson. On est dans le même ordre d'idée
* oov_ratio 0.0
    * On a pas de pairs de mots inconnus

### h. De même, en vous inspirant de la documentation, évaluez le modèle sur les données de test appelées questions-words.txt. Pouvez-vous expliquer ce que mesure ce test ? Les résultats du modèle sont-ils satisfaisants ? Commentez.

In [34]:
analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
analogy_scores

(0.7401448525607863,
 [{'section': 'capital-common-countries',
   'correct': [('ATHENS', 'GREECE', 'BANGKOK', 'THAILAND'),
    ('ATHENS', 'GREECE', 'BEIJING', 'CHINA'),
    ('ATHENS', 'GREECE', 'BERLIN', 'GERMANY'),
    ('ATHENS', 'GREECE', 'BERN', 'SWITZERLAND'),
    ('ATHENS', 'GREECE', 'CAIRO', 'EGYPT'),
    ('ATHENS', 'GREECE', 'CANBERRA', 'AUSTRALIA'),
    ('ATHENS', 'GREECE', 'HAVANA', 'CUBA'),
    ('ATHENS', 'GREECE', 'HELSINKI', 'FINLAND'),
    ('ATHENS', 'GREECE', 'ISLAMABAD', 'PAKISTAN'),
    ('ATHENS', 'GREECE', 'MADRID', 'SPAIN'),
    ('ATHENS', 'GREECE', 'MOSCOW', 'RUSSIA'),
    ('ATHENS', 'GREECE', 'OSLO', 'NORWAY'),
    ('ATHENS', 'GREECE', 'OTTAWA', 'CANADA'),
    ('ATHENS', 'GREECE', 'PARIS', 'FRANCE'),
    ('ATHENS', 'GREECE', 'ROME', 'ITALY'),
    ('ATHENS', 'GREECE', 'STOCKHOLM', 'SWEDEN'),
    ('ATHENS', 'GREECE', 'TEHRAN', 'IRAN'),
    ('ATHENS', 'GREECE', 'TOKYO', 'JAPAN'),
    ('BAGHDAD', 'IRAQ', 'BANGKOK', 'THAILAND'),
    ('BAGHDAD', 'IRAQ', 'BEIJING', 'CHINA'

Score d'analogie: 0.7401448525607863

Cette mesure est probablement entre 0 et 1. Donc on a une "accuracy" de 0.74

## 2. Entraîner et tester deux nouveaux modèles à partir de corpus

In [35]:
import gensim.downloader

corpus = gensim.downloader.load('text8')

### a. Combien de phrases et de mots possède ce corpus ?

In [36]:
print('Number of docs:', len(list(corpus)))


Number of docs: 1701


In [37]:
from gensim.models import Word2Vec
import time
exec_start = time.time()

model = Word2Vec(corpus, workers=mp.cpu_count())

print(model)

w2vb_vectors = model.wv
# del model
print('time elapsed: ', (time.time() - exec_start), ' seconds')

Word2Vec(vocab=71290, size=100, alpha=0.025)
time elapsed:  62.63047122955322  seconds


In [38]:
print('State: ', len(w2vb_vectors['state']))

State:  100


Dimension: 100

Taille du vocabulaire: 71290

### b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus. (Procédez éventuellement progressivement, en commençant par 1%, puis 10% du corpus, pour contrôler le temps.) Combien de temps prend l’entraînement et quelle est la taille (en Mo) du modèle word2vec résultant ?

In [39]:
import sys

print('Vecteur', sys.getsizeof(w2vb_vectors['state']))
print('Une valeur du vecteur', sys.getsizeof(w2vb_vectors['state'][0]))
print("L'objet contenant", sys.getsizeof(w2vb_vectors))

Vecteur 96
Une valeur du vecteur 28
L'objet contenant 64


Si un seule valeur du vecteur fait 28 bytes (admettons), la dimension étant de 100, alors taille du vecteur * 71290 + 64 (64 étant la taille de l'objet de liste w2vb_vectors)

In [40]:
print(((28 * 100 + 96) * 71290 + 64) / 1024 / 1024, ' Mo')

196.89169311523438  Mo


### c. Quelle dimension avez-vous choisi pour le plongement (embedding) de ce nouveau modèle ?

La dimension est de 100

### d. Testez quantitativement la qualité de ce nouveau modèle comme dans la partie 1 (points h et i). Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle serait la raison de la différence ?

In [41]:
from gensim.test.utils import datapath

In [42]:
similarities = w2vb_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

similarities

((0.6099347877508037, 3.840246974484307e-37),
 SpearmanrResult(correlation=0.6219077968382327, pvalue=5.955491702677223e-39),
 0.56657223796034)

* Pearson: 0.60 (0.62 précédent)
* Spearman 0.62 (0.65 précédent)
* 0.56657223796034 au lieu de 0.0. Moins bien aussi

De manière générale, le modèle précédent était meilleure

Précision: en appelant model.train, on a reentrainé le model. Ce qui est normalement fait par défaut. De base nos mesures de correlation étaient plus faibles. Autour de 0.3

In [43]:
analogy_scores = w2vb_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
analogy_scores

(0.23587816233802658,
 [{'section': 'capital-common-countries',
   'correct': [('ATHENS', 'GREECE', 'BERLIN', 'GERMANY'),
    ('ATHENS', 'GREECE', 'MOSCOW', 'RUSSIA'),
    ('ATHENS', 'GREECE', 'PARIS', 'FRANCE'),
    ('BAGHDAD', 'IRAQ', 'BERLIN', 'GERMANY'),
    ('BAGHDAD', 'IRAQ', 'HANOI', 'VIETNAM'),
    ('BAGHDAD', 'IRAQ', 'HAVANA', 'CUBA'),
    ('BAGHDAD', 'IRAQ', 'PARIS', 'FRANCE'),
    ('BAGHDAD', 'IRAQ', 'STOCKHOLM', 'SWEDEN'),
    ('BAGHDAD', 'IRAQ', 'TEHRAN', 'IRAN'),
    ('BAGHDAD', 'IRAQ', 'ATHENS', 'GREECE'),
    ('BANGKOK', 'THAILAND', 'BERLIN', 'GERMANY'),
    ('BANGKOK', 'THAILAND', 'CAIRO', 'EGYPT'),
    ('BANGKOK', 'THAILAND', 'MADRID', 'SPAIN'),
    ('BANGKOK', 'THAILAND', 'MOSCOW', 'RUSSIA'),
    ('BANGKOK', 'THAILAND', 'PARIS', 'FRANCE'),
    ('BANGKOK', 'THAILAND', 'TEHRAN', 'IRAN'),
    ('BANGKOK', 'THAILAND', 'ATHENS', 'GREECE'),
    ('BEIJING', 'CHINA', 'BERLIN', 'GERMANY'),
    ('BEIJING', 'CHINA', 'HAVANA', 'CUBA'),
    ('BEIJING', 'CHINA', 'KABUL', 'AFGHANIST

Score d'analogie de 0.32 (0.74 précédent). Largement inférieur

### e. Considérez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters (corpus de 413 Mo que vous trouverez en ligne comme wikipedia_augmented.dat). Utilisez ce nouveau corpus afin d’entraîner un nouveau modèle word2vec. Vous devez pour cela utiliser la fonction Text8Corpus() afin de charger et prétraiter le texte du corpus (tokenization et segmentation en phrases, qui étaient déjà faites par la fonction load() du downloader). Combien de temps prend l’entraînement et quelle est la taille (en Mo) du modèle word2vec résultant ? Quelle est la dimension de l’embedding ?

In [44]:
from gensim.models import word2vec

sentences = word2vec.Text8Corpus('./wikipedia_augmented.dat')


In [45]:
import multiprocessing as mp
import time

exec_start = time.time()

model = word2vec.Word2Vec(sentences, workers=mp.cpu_count())

print(model)

w2vc_vectors = model.wv
# del model
print('time elapsed: ', (time.time() - exec_start), ' seconds')

Word2Vec(vocab=124372, size=100, alpha=0.025)
time elapsed:  250.58165645599365  seconds


In [46]:
import sys

print('Vecteur', sys.getsizeof(w2vc_vectors['state']))
print('Une valeur du vecteur', sys.getsizeof(w2vc_vectors['state'][0]))
print("L'objet contenant", sys.getsizeof(w2vc_vectors))

Vecteur 96
Une valeur du vecteur 28
L'objet contenant 64


In [47]:
print(((28 * 100 + 96) * 124372 + 64) / 1024 / 1024, ' Mo')

343.4957275390625  Mo


La dimension est de 100 (Valeur par défaut)

### f. Testez aussi ce modèle. Est-il meilleur que le précédent ? Pour quelle raison ?

In [48]:
from gensim.test.utils import datapath

In [49]:
similarities = w2vc_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

similarities

((0.4916134686691972, 7.035715295208388e-23),
 SpearmanrResult(correlation=0.49640985859069087, pvalue=2.3194103644935318e-23),
 0.0)

La première fois que j'ai executé le précédent, il l'était. Cette fois étrangement non.

C'est difficile à imaginer une raison plausible. Peut-être que cette extension, en ajoutant des mots, diminue leur correlation entre eux?

In [50]:
analogy_scores = w2vc_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
analogy_scores

(0.2891015879921688,
 [{'section': 'capital-common-countries',
   'correct': [('ATHENS', 'GREECE', 'BEIJING', 'CHINA'),
    ('ATHENS', 'GREECE', 'BERLIN', 'GERMANY'),
    ('ATHENS', 'GREECE', 'HAVANA', 'CUBA'),
    ('ATHENS', 'GREECE', 'MADRID', 'SPAIN'),
    ('ATHENS', 'GREECE', 'MOSCOW', 'RUSSIA'),
    ('ATHENS', 'GREECE', 'OTTAWA', 'CANADA'),
    ('ATHENS', 'GREECE', 'PARIS', 'FRANCE'),
    ('ATHENS', 'GREECE', 'ROME', 'ITALY'),
    ('ATHENS', 'GREECE', 'TEHRAN', 'IRAN'),
    ('ATHENS', 'GREECE', 'TOKYO', 'JAPAN'),
    ('BAGHDAD', 'IRAQ', 'BEIJING', 'CHINA'),
    ('BAGHDAD', 'IRAQ', 'BERLIN', 'GERMANY'),
    ('BAGHDAD', 'IRAQ', 'CANBERRA', 'AUSTRALIA'),
    ('BAGHDAD', 'IRAQ', 'HANOI', 'VIETNAM'),
    ('BAGHDAD', 'IRAQ', 'HAVANA', 'CUBA'),
    ('BAGHDAD', 'IRAQ', 'KABUL', 'AFGHANISTAN'),
    ('BAGHDAD', 'IRAQ', 'MADRID', 'SPAIN'),
    ('BAGHDAD', 'IRAQ', 'OTTAWA', 'CANADA'),
    ('BAGHDAD', 'IRAQ', 'PARIS', 'FRANCE'),
    ('BAGHDAD', 'IRAQ', 'ROME', 'ITALY'),
    ('BAGHDAD', 'IRAQ',

Les résultats comparés à la partie précédente:
* mieux que la première fois
* moins bons qu'après réentraînement.


### g. Créez un nouveau fichier de test en augmentant questions-words.txt avec une catégorie de mots de votre choix (environ 20 items de test). Par exemple, à partir de {(eye, see), (ear, listen), (foot, walk)} on peut construire 6 items de test. Testez les trois modèles sur ces données, présentez et commentez vos résultats.

In [54]:
print(w2v_vectors)
print(w2vb_vectors)
print(w2vc_vectors)

In [58]:
analogy_scores1 = w2v_vectors.evaluate_word_analogies('questions-words2.txt')
analogy_scores2 = w2vb_vectors.evaluate_word_analogies('questions-words2.txt')
analogy_scores3 = w2vc_vectors.evaluate_word_analogies('questions-words2.txt')

print('1', analogy_scores1, '\n')
print('2', analogy_scores2, '\n')
print('3', analogy_scores3, '\n')

1 (0.0, [{'section': 'custom', 'correct': [], 'incorrect': [('EYE', 'SEE', 'EAR', 'LISTEN'), ('EYE', 'SEE', 'HAND', 'TOUCH'), ('EYE', 'SEE', 'NOSE', 'SMELL'), ('EYE', 'SEE', 'TONGUE', 'TASTE'), ('EAR', 'LISTEN', 'EYE', 'SEE'), ('EAR', 'LISTEN', 'HAND', 'TOUCH'), ('EAR', 'LISTEN', 'NOSE', 'SMELL'), ('EAR', 'LISTEN', 'TONGUE', 'TASTE'), ('HAND', 'TOUCH', 'EYE', 'SEE'), ('HAND', 'TOUCH', 'EAR', 'LISTEN'), ('HAND', 'TOUCH', 'HAND', 'TOUCH'), ('HAND', 'TOUCH', 'NOSE', 'SMELL'), ('NOSE', 'SMELL', 'EYE', 'SEE'), ('NOSE', 'SMELL', 'EAR', 'LISTEN'), ('NOSE', 'SMELL', 'HAND', 'TOUCH'), ('NOSE', 'SMELL', 'TONGUE', 'TASTE'), ('TONGUE', 'TASTE', 'EYE', 'SEE'), ('TONGUE', 'TASTE', 'HAND', 'TOUCH'), ('TONGUE', 'TASTE', 'NOSE', 'SMELL'), ('TONGUE', 'TASTE', 'EAR', 'LISTEN')]}, {'section': 'Total accuracy', 'correct': [], 'incorrect': [('EYE', 'SEE', 'EAR', 'LISTEN'), ('EYE', 'SEE', 'HAND', 'TOUCH'), ('EYE', 'SEE', 'NOSE', 'SMELL'), ('EYE', 'SEE', 'TONGUE', 'TASTE'), ('EAR', 'LISTEN', 'EYE', 'SEE'), ('

Avec cette augmentation, on a pas un score très élevé. :(

Seul le second modèle s'en sort on dirait.